In [1]:
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
os.chdir('../..')

os.environ["OMP_NUM_THREADS"] = "4"

In [38]:
from pathlib import Path

from hydra import compose, initialize

import pandas as pd
import torch 
import numpy as np 
from omegaconf import DictConfig, OmegaConf
from src.models.recbole import RecboleBench
from src.preprocessing import ClassicDataset
from src.utils.logging import get_logger
from src.utils.processing import save_results, train_test_split, pandas_to_recbole
from src.utils.metrics import run_all_metrics, coverage
from src.utils.processing import pandas_to_aggregate
logger = get_logger(name=__name__)

%load_ext autoreload
%autoreload 2
np.float = float

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Import recbole Reqired Packages
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.utils import init_seed, init_logger
from recbole.quick_start import run_recbole

2023-11-22 16:49:38,175	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-11-22 16:49:38,277	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [4]:
with initialize(config_path='../../config', version_base=None):
    cfg = compose(config_name='config')

In [39]:
"""RecBole module"""
from pathlib import Path
import os

from omegaconf import DictConfig

import omegaconf
import pandas as pd
import shutil

from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.utils import init_seed

from src.models.recbole import RecboleBench
from src.preprocessing import ClassicDataset
from src.utils.logging import get_logger
from src.utils.processing import data_split, save_results
from src.utils.metrics import run_all_metrics, coverage


logger = get_logger(name=__name__)

In [6]:
# load configs
recbole_name: str = cfg["library"]["name"]
cfg_data = cfg["dataset"]
cfg_model = cfg["library"]["recbole_model"]

# split data into samples
dataset_folder = Path(os.path.join("preproc_data", cfg_data["name"]))
dataset = ClassicDataset()
dataset.prepare(cfg_data)

if (
    dataset_folder.joinpath("train.parquet").exists() and
    dataset_folder.joinpath("test.parquet").exists()
):
    data_train = pd.read_parquet(dataset_folder.joinpath("train.parquet"))
    data_test = pd.read_parquet(dataset_folder.joinpath("test.parquet"))
else:
    data_train, data_test = train_test_split(
        dataset.prepared_data,
        test_size=cfg_data["splitting"]["test_size"],
        splitting_type=cfg_data["splitting"]["strategy"],
    )
    data_train.to_parquet(dataset_folder.joinpath("train.parquet"))
    data_test.to_parquet(dataset_folder.joinpath("test.parquet"))


In [7]:
pandas_to_recbole(dataset=data_train, dataset_name=cfg_data["name"], split_name="train")
pandas_to_recbole(dataset=data_test, dataset_name=cfg_data["name"], split_name="test")

parameter_dict = OmegaConf.to_container(cfg_model["recbole_params"], resolve=True)
parameter_dict.update(dict(
    data_path = os.path.join("data", "tmp"),
    dataset = cfg_data["name"],
))

config = Config(
    model=cfg_model["name"],
    dataset=cfg_data["name"],
    config_file_list=None,
    config_dict=parameter_dict,
)

init_seed(config["seed"], config["reproducibility"])

dataset = create_dataset(config)
train_set, _, test_set = data_preparation(config, dataset)

model_folder = dataset_folder.joinpath(recbole_name, cfg_model["name"])

if cfg_model["saved_model"]:
    bench = RecboleBench.initialize_saved_model(
        model_folder.joinpath(cfg_model["saved_model_name"]), train_set
    )
else:
    bench = None

if bench is None:
    if cfg_model["enable_optimization"]:
        data_train_opt, data_val = train_test_split(
            data_train,
            test_size=cfg_data["splitting"]["val_size"],
            splitting_type=cfg_data["splitting"]["strategy"],
        )
        pandas_to_recbole(dataset=data_train_opt, dataset_name=cfg_data["name"], split_name="train_opt")
        pandas_to_recbole(dataset=data_val, dataset_name=cfg_data["name"], split_name="valid")
        config['benchmark_filename'] = ['train_opt', 'valid', 'valid']
        dataset = create_dataset(config)
        train_opt_loader, _, valid_loader = data_preparation(config, dataset)
        
        bench = RecboleBench.initialize_with_optimization(
            cfg_model["optuna_optimizer"],
            train_opt_loader,
            valid_loader,
            data_val,
        )
        was_optimized = True
    else:
        bench = RecboleBench.initialize_with_params(
            train_loader=train_set,
            model_params=cfg_model['model']
        )
        was_optimized = False

bench = RecboleBench.initialize_with_params(
    train_loader=train_set, model_params=bench.config.final_config_dict
)
bench.fit(train_set)
bench.save_model(model_folder)

top_100_items = bench.recommend_k(test_set, k=100)

Max value of user's history interaction records has reached 42.77929155313351% of the total.
Max value of user's history interaction records has reached 42.77929155313351% of the total.
Train   299: 100%|███████████████████████████| 1/1 [00:00<00:00,  2.81it/s, GPU RAM: 0.05 G/47.45 G]


NameError: name 'all_user_loader' is not defined

In [29]:
train_set.user_list

The batch_size of interaction: 1079
    user_id, torch.Size([1079]), cpu, torch.int64


In [35]:
test_set.dataset.user_num

1079

In [37]:
torch.arange(1, test_set.dataset.user_num)

tensor([   1,    2,    3,  ..., 1076, 1077, 1078])

In [10]:
user_list = test_set.uid_list.clone().detach()

In [30]:
torch.tensor([3,    4,    5,])

tensor([3, 4, 5])

In [ ]:
from recbole.utils.case_study import full_sort_topk
import torch
topk = full_sort_topk(torch.tensor([1079]), bench.model, test_set, 100)[1]

In [14]:
topk.shape

torch.Size([814, 100])

In [15]:
train_set

In [40]:
top_100_items = bench.recommend_k(test_set, k=100)

Generating top_100 recommendations: 1 batch [00:00, 59.46 batch/s]


In [42]:
top_100_items

array([[  7,   6,  31, ...,  78,  89, 274],
       [ 16,  87,  29, ...,  93,  68, 241],
       [ 91,  46,  35, ..., 162, 135,   1],
       ...,
       [ -1,  -1,  -1, ...,  -1,  -1,  -1],
       [ -1,  -1,  -1, ...,  -1,  -1,  -1],
       [ -1,  -1,  -1, ...,  -1,  -1,  -1]])

In [43]:
pandas_to_aggregate(data_test).shape

(814,)

In [19]:
data_train.userId.max()

1077

In [23]:
data_train.userId.nunique()

1078

In [44]:
metrics = run_all_metrics(top_100_items, pandas_to_aggregate(data_test), [5, 10, 20, 100])
coverage_metrics = []
for k in (5, 10, 20, 100):
    coverage_metrics.append(coverage(top_100_items, data_test['itemId'].nunique(), k))

metrics_df = pd.DataFrame(metrics, index=[5, 10, 20, 100], columns=(
    'Precision@k', 'Recall@K', 'MAP@K', 'nDCG@k', 'MRR@k', 'HitRate@k'
))
metrics_df['Coverage@K'] = coverage_metrics

metrics_df['Time_fit'] = bench.learning_time
metrics_df['Time_predict'] = bench.predict_time

In [45]:
metrics_df

,Precision@k,Recall@K,MAP@K,nDCG@k,MRR@k,HitRate@k,Coverage@K,Time_fit,Time_predict
5,0.014947,0.011822,0.006767,0.013666,0.027416,0.061425,0.223529,126.414623,None
10,0.025809,0.024840,0.007705,0.019339,0.035146,0.122850,0.291176,126.414623,None
20,0.049040,0.049008,0.009545,0.029347,0.041703,0.217445,0.402941,126.414623,None
100,0.269203,0.269203,0.017257,0.091309,0.050487,0.584767,0.791176,126.414623,None


In [8]:
config = Config(
            model=cfg_model["name"],
            dataset=cfg_data["name"],
            config_file_list=None,
            config_dict=parameter_dict,
        )

init_seed(config["seed"], config["reproducibility"])

dataset = create_dataset(config)

train_set, valid_set, test_set = data_preparation(config, dataset)

model_folder = Path(
    "/".join(
        ("preproc_data", cfg_data["name"], recbole_name, cfg_model["name"])
    )
)


In [9]:
torch.cuda.set_device(0)
torch.cuda.empty_cache()

In [ ]:
bench = RecboleBench.initialize_with_params(
                    train_loader=train_set,
                    model_params=cfg_model['model'])
                
config["benchmark_filename"] = ["trainval", "val", "test"]
dataset = create_dataset(config)
trainval_set, valid_set, test_set = data_preparation(config, dataset)
bench = RecboleBench.initialize_with_params(
    train_loader=trainval_set, model_params=bench.config.final_config_dict
)
bench.fit(trainval_set)

In [16]:
ranks = bench.get_relevant_ranks(test_set)
top_100_items = bench.recommend_k(test_set, 100)

metrics = run_all_metrics(ranks, [5, 10, 20, 100])
coverage_metrics = []
for k in (5, 10, 20, 100):
    coverage_metrics.append(
        coverage(top_100_items, test_set.dataset.item_num, k)
    )

metrics_df = pd.DataFrame(
    metrics,
    index=[5, 10, 20, 100],
    columns=(
        "Precision@k",
        "Recall@K",
        "MAP@K",
        "nDCG@k",
        "MRR@k",
        "HitRate@k",
    ),
)
metrics_df["Coverage@K"] = coverage_metrics

metrics_df["Time_fit"] = bench.learning_time
metrics_df["Time_predict"] = bench.predict_time
metrics_df

Calculating item ranks: 1 batch [00:00, 61.58 batch/s]
Generating top_100 recommendations: 0 batch [00:00, ? batch/s]/home/user/conda/lib/python3.9/site-packages/recbole/utils/case_study.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  uid_series = torch.tensor(uid_series)
Generating top_100 recommendations: 1 batch [00:00, 87.55 batch/s]
21 Nov 12:37    WARNING  Some of the necessary parameters are absent! Calculating coverage, diversity and novelty is impossible and will not be performed.


,Precision@k,Recall@K,MAP@K,nDCG@k,MRR@k,HitRate@k,Coverage@K,Time_fit,Time_predict
5,0.157299,0.112932,0.093005,0.153463,0.273036,0.471464,0.193460,146.592203,0.035511
10,0.207018,0.195530,0.090158,0.173592,0.294323,0.630273,0.267030,146.592203,0.035511
20,0.328139,0.326636,0.104553,0.222209,0.304581,0.777916,0.367847,146.592203,0.035511
100,0.755596,0.755596,0.138693,0.354835,0.309760,0.962779,0.708447,146.592203,0.035511


In [10]:
parameter_dict['metrics'] = ['Precision', 'Recall', 'MAP', 'NDCG', 'MRR', 'HIT', 'ItemCoverage']

In [18]:
parameter_dict

{'USER_ID_FIELD': 'user_id',
 'ITEM_ID_FIELD': 'item_id',
 'RATING_FIELD': 'rating',
 'TIME_FIELD': 'timestamp',
 'benchmark_filename': ['train', 'val', 'test'],
 'load_col': {'inter': ['user_id', 'item_id', 'rating']},
 'worker': 8,
 'nproc': 4,
 'gpu_id': 0,
 'train_batch_size': 4096,
 'eval_batch_size': 40960000,
 'neg_sampling': None,
 'train_neg_sample_args': None,
 'epochs': 300,
 'show_progress': True,
 'data_path': 'data/tmp',
 'dataset': 'foursquare',
 'metrics': ['Precision',
  'Recall',
  'MAP',
  'NDCG',
  'MRR',
  'HIT',
  'ItemCoverage']}

In [12]:
# Run recbole
results = run_recbole(model=config['model'], dataset=config['dataset'], config_dict=parameter_dict)

21 Nov 12:33    INFO  ['/home/user/conda/lib/python3.9/site-packages/ipykernel_launcher.py', '--ip=127.0.0.1', '--stdin=9018', '--control=9016', '--hb=9015', '--Session.signature_scheme="hmac-sha256"', '--Session.key=b"99170925-ba07-4d35-8dbb-8a2906b328a7"', '--shell=9017', '--transport="tcp"', '--iopub=9019', '--f=/root/.local/share/jupyter/runtime/kernel-v2-348xc5llXntPsZQ.json']
21 Nov 12:33    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = data/tmp/foursquare
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 4096
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 

### Compare recbole buildin training and evaluation and our pipeline

In [17]:
metrics_df.loc[10]

Precision@k       0.207018
Recall@K          0.195530
MAP@K             0.090158
nDCG@k            0.173592
MRR@k             0.294323
HitRate@k         0.630273
Coverage@K        0.267030
Time_fit        146.592203
Time_predict      0.035511
Name: 10, dtype: float64

In [14]:
results['test_result']

OrderedDict([('precision@10', 0.0971),
             ('recall@10', 0.1619),
             ('map@10', 0.074),
             ('ndcg@10', 0.1453),
             ('mrr@10', 0.2477),
             ('hit@10', 0.5533),
             ('itemcoverage@10', 0.1717)])

### After training lets make final fit on trainval set

In [ ]:
config['benchmark_filename'] = ['trainval', 'val', 'test']

In [ ]:
dataset = create_dataset(config)

In [ ]:
trainval_set, valid_set, test_set = data_preparation(config, dataset)

18 Aug 11:33    INFO  [Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]
18 Aug 11:33    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [0.8, 0.1, 0.1]}, 'group_by': 'user', 'order': 'RO', 'mode': 'full'}]


In [ ]:
model = RecboleBench.initialize_with_params(
                    train_loader=trainval_set)

18 Aug 11:33    WARNING  Max value of user's history interaction records has reached 42.77929155313351% of the total.


In [ ]:
model.fit(trainval_set)

18 Aug 11:33    INFO  epoch 0 training [time: 0.01s, train loss: 254.0525]
18 Aug 11:33    INFO  epoch 1 training [time: 0.01s, train loss: 253.2876]
18 Aug 11:33    INFO  epoch 2 training [time: 0.01s, train loss: 252.1098]
18 Aug 11:33    INFO  epoch 3 training [time: 0.01s, train loss: 250.2143]
18 Aug 11:33    INFO  epoch 4 training [time: 0.01s, train loss: 247.4650]
18 Aug 11:33    INFO  epoch 5 training [time: 0.01s, train loss: 243.8431]
18 Aug 11:33    INFO  epoch 6 training [time: 0.01s, train loss: 239.6220]
18 Aug 11:33    INFO  epoch 7 training [time: 0.01s, train loss: 235.5777]
18 Aug 11:33    INFO  epoch 8 training [time: 0.01s, train loss: 232.8785]
18 Aug 11:33    INFO  epoch 9 training [time: 0.01s, train loss: 232.3856]
18 Aug 11:33    INFO  epoch 10 training [time: 0.01s, train loss: 232.9111]
18 Aug 11:33    INFO  epoch 11 training [time: 0.01s, train loss: 232.8951]
18 Aug 11:33    INFO  epoch 12 training [time: 0.01s, train loss: 232.1133]
18 Aug 11:33    INFO  

In [ ]:
ranks = model.get_relevant_ranks(test_set)
top_100_items = model.recommend_k(test_set, 100)
metrics = run_all_metrics(ranks, [5, 10, 20, 100])
coverage_metrics = []
for k in (5, 10, 20, 100):
    coverage_metrics.append(coverage(
        top_100_items,
        test_set.dataset.item_num,
        k
    ))

metrics_df = pd.DataFrame(metrics, index=[5, 10, 20, 100], columns=(
    'Precision@k', 'Recall@K', 'MAP@K', 'nDCG@k', 'MRR@k', 'HitRate@k'
))
metrics_df['Coverage@K'] = coverage_metrics

metrics_df['Time_fit'] = model.learning_time
metrics_df['Time_predict'] = model.predict_time
metrics_df

/home/recsys/miniconda/envs/rec_env/lib/python3.9/site-packages/recbole/utils/case_study.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  uid_series = torch.tensor(uid_series)


  0%|          | 0/4 [00:00<?, ?it/s]

,Precision@k,Recall@K,MAP@K,nDCG@k,MRR@k,HitRate@k,Coverage@K,Time_fit,Time_predict
5,0.157113,0.113496,0.093540,0.151451,0.261042,0.450372,0.337875,6.034429,0.048151
10,0.209672,0.197785,0.090865,0.173552,0.284857,0.630273,0.455041,6.034429,0.048151
20,0.324464,0.322734,0.104331,0.220295,0.295406,0.777916,0.604905,6.034429,0.048151
100,0.749474,0.749474,0.137642,0.351021,0.300620,0.960298,0.942779,6.034429,0.048151
